## Getting started with Spark dataframes.

To start working with Spark dataframes, we first need to start a Spark session.

In [1]:
import findspark

In [2]:
findspark.init('/home/sushant/spark-2.1.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('haveFun').getOrCreate()

In [5]:
df1 = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [6]:
df1

DataFrame[age: bigint, name: string]

In [8]:
df1.show()
# Nothings takes place till we apply an 'action'. 

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [9]:
df1.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



The above function prints the schema of the table. It tells that there are two columns -- age and name and their type.

In [10]:
df1.columns

['age', 'name']

In [11]:
df1.describe()

DataFrame[summary: string, age: string, name: string]

In [12]:
df1.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



describe: statistical summary of numeric data columns.

### Import type tools to handle schema properly. 

In [13]:
from pyspark.sql.types import (StructField, IntegerType, 
                              StringType, StructType)

Now create a list of StructFields. StructFields take in three parameters -- name, datatype, and if nulllable.

In [18]:
data_schema = [StructField('age', IntegerType(), True), 
              StructField('name', StringType(), True)]
# the second argument is class instance. 
# the third fields tells if null values are ok or not.
# This is a schema that we are expecting.

In [19]:
final_struc = StructType(fields=data_schema)

In [20]:
df2 = spark.read.json('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json', schema = final_struc)

In [21]:
df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Extracting columns

In [22]:
df2['age']

Column<b'age'>

In [23]:
type(df2['age'])

pyspark.sql.column.Column

df2['age'] gives a column object.

In [25]:
df2.select('age')

DataFrame[age: int]

In [26]:
df2.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [28]:
type(df2.select('age'))

pyspark.sql.dataframe.DataFrame

In [37]:
# Selecting multiple columns
df2.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



df2.select gives a dataframe.

Extracting rows

In [30]:
df2.head(2)
# gives a list of rows

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [31]:
df2.head(2)[1]

Row(age=30, name='Andy')

In [35]:
type(df2.head())
# df2.head() gives a row object. 

pyspark.sql.types.Row

So far we have seen a column object, a row object, and of course a dataframe object. Specialized objects allow distributed computing.  

#### Adding new columns

In [41]:
df2.withColumn('doubleAge', 2.0 * df2['age']).show()

+----+-------+---------+
| age|   name|doubleAge|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|     60.0|
|  19| Justin|     38.0|
+----+-------+---------+



In [43]:
df2.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



withColumn is not an inplace operation. 

We can also rename a column. 

In [45]:
df2.withColumnRenamed('age', 'newAge').show()

+------+-------+
|newAge|   name|
+------+-------+
|  null|Michael|
|    30|   Andy|
|    19| Justin|
+------+-------+



In [48]:
df2.show()
# withColumnRenamed is also not an inplace operation.

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Using pure SQL on Spark.

In [49]:
df2.createOrReplaceTempView('people')

In [50]:
results = spark.sql("SELECT * FROM people")

In [51]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [57]:
results2 = spark.sql("SELECT * FROM people WHERE name = 'Justin'")

In [58]:
results2.show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
+---+------+



### Basic operations with data frames. 

In [62]:
df3  = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv', inferSchema=True, header=True)

In [63]:
df3.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [64]:
df3.show()

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902

In [67]:
df3.head(2)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002)]

In [69]:
df3.filter("Close < 500").show()

+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|                Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+--------------------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902

In [70]:
df3.filter("Close < 500").select(['Date', 'open']).show()

+--------------------+------------------+
|                Date|              open|
+--------------------+------------------+
|2010-01-04 00:00:...|        213.429998|
|2010-01-05 00:00:...|        214.599998|
|2010-01-06 00:00:...|        214.379993|
|2010-01-07 00:00:...|            211.75|
|2010-01-08 00:00:...|        210.299994|
|2010-01-11 00:00:...|212.79999700000002|
|2010-01-12 00:00:...|209.18999499999998|
|2010-01-13 00:00:...|        207.870005|
|2010-01-14 00:00:...|210.11000299999998|
|2010-01-15 00:00:...|210.92999500000002|
|2010-01-19 00:00:...|        208.330002|
|2010-01-20 00:00:...|        214.910006|
|2010-01-21 00:00:...|        212.079994|
|2010-01-22 00:00:...|206.78000600000001|
|2010-01-25 00:00:...|202.51000200000001|
|2010-01-26 00:00:...|205.95000100000001|
|2010-01-27 00:00:...|        206.849995|
|2010-01-28 00:00:...|        204.930004|
|2010-01-29 00:00:...|        201.079996|
|2010-02-01 00:00:...|192.36999699999998|
+--------------------+------------

But instead of SQL like statements we can also use Python's own comparison operators. 

In [71]:
df3.filter(df3['Close'] < 500).select(['Date', 'open']).show()

+--------------------+------------------+
|                Date|              open|
+--------------------+------------------+
|2010-01-04 00:00:...|        213.429998|
|2010-01-05 00:00:...|        214.599998|
|2010-01-06 00:00:...|        214.379993|
|2010-01-07 00:00:...|            211.75|
|2010-01-08 00:00:...|        210.299994|
|2010-01-11 00:00:...|212.79999700000002|
|2010-01-12 00:00:...|209.18999499999998|
|2010-01-13 00:00:...|        207.870005|
|2010-01-14 00:00:...|210.11000299999998|
|2010-01-15 00:00:...|210.92999500000002|
|2010-01-19 00:00:...|        208.330002|
|2010-01-20 00:00:...|        214.910006|
|2010-01-21 00:00:...|        212.079994|
|2010-01-22 00:00:...|206.78000600000001|
|2010-01-25 00:00:...|202.51000200000001|
|2010-01-26 00:00:...|205.95000100000001|
|2010-01-27 00:00:...|        206.849995|
|2010-01-28 00:00:...|        204.930004|
|2010-01-29 00:00:...|        201.079996|
|2010-02-01 00:00:...|192.36999699999998|
+--------------------+------------

In [74]:
df3.filter((df3['Close'] < 500) & ~(df3['Open'] < 200)).show(4)
# Note: we gotta enclose conditions in paranthesis and use '&' not 'and'

+--------------------+----------+----------+------------------+----------+---------+------------------+
|                Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+--------------------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04 00:00:...|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|    211.75|212.000006|        209.050005|    210.58|119282800|          27.28265|
+--------------------+----------+----------+------------------+----------+---------+------------------+
only showing top 4 rows



In [76]:
df3.filter(df3['High'] == 215.23).show()

+--------------------+----------+------+----------+----------+---------+------------------+
|                Date|      Open|  High|       Low|     Close|   Volume|         Adj Close|
+--------------------+----------+------+----------+----------+---------+------------------+
|2010-01-06 00:00:...|214.379993|215.23|210.750004|210.969995|138040000|27.333178000000004|
+--------------------+----------+------+----------+----------+---------+------------------+



Note how show is like print(). No output generated. To store the output to other variable use collect. 

In [77]:
res = df3.filter(df3['High'] == 215.23).collect()

In [78]:
res

[Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [79]:
type(res)

list

In [81]:
len(res)

1

In [82]:
res[0]

Row(Date=datetime.datetime(2010, 1, 6, 0, 0), Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)

In [86]:
resUseful = res[0].asDict()

In [88]:
resUseful

{'Adj Close': 27.333178000000004,
 'Close': 210.969995,
 'Date': datetime.datetime(2010, 1, 6, 0, 0),
 'High': 215.23,
 'Low': 210.750004,
 'Open': 214.379993,
 'Volume': 138040000}

In [89]:
resUseful['Adj Close']

27.333178000000004

### Group by and aggregate functions

In [95]:
df4 = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/sales_info.csv', 
                     inferSchema=True, header=True)

In [99]:
df4.show(10)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
+-------+-------+-----+
only showing top 10 rows



In [98]:
df4.describe().show()

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|     12|     12|                12|
|   mean|   null|   null| 360.5833333333333|
| stddev|   null|   null|250.08742410799007|
|    min|   APPL|  Chris|             120.0|
|    max|   MSFT|Vanessa|             870.0|
+-------+-------+-------+------------------+



In [102]:
df4.groupBy('Company').max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [103]:
df4.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [107]:
df4.agg({'Company': 'min'}).show()

+------------+
|min(Company)|
+------------+
|        APPL|
+------------+



In [111]:
df4.agg({'Sales': 'min'}).show()

+----------+
|min(Sales)|
+----------+
|     120.0|
+----------+



In [112]:
grouped_data = df4.groupBy('Company')

In [114]:
grouped_data.agg({'Sales': 'max'}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



Using functions

In [115]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [116]:
df4.select(avg('Sales')).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [118]:
df4.select(countDistinct('Sales').alias('Distinct entries')).show()

+----------------+
|Distinct entries|
+----------------+
|              11|
+----------------+



In [119]:
df4.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [120]:
from pyspark.sql.functions import format_number

In [122]:
df4.select(format_number(stddev('Sales'), 2).alias('std')).show()

+------+
|   std|
+------+
|250.09|
+------+



Using orderBy

In [124]:
df4.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [126]:
df4.orderBy(df4['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



## Missing data

In [127]:
!pwd

/home/sushant/Documents/SparkUdemyCourse


In [131]:
df5 = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_DataFrames/ContainsNull.csv', 
                     inferSchema=True, header=True)

In [132]:
df5.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [134]:
df5.dropna().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [137]:
df5.na.drop().show()
# drops any row that contains any amount of missing data

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [138]:
df5.na.drop(thresh=2).show()
# Row needs to have at least 2 non-null entries, otherwise it wil be dropped.  

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [141]:
df5.na.drop(how = 'any').show()
# drops row which has a null value anywhere

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [142]:
df5.na.drop(how = 'all').show()
# drops row only if all the values are null (here we have non-null id for all of them)

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [145]:
df5.na.drop(subset = ['Sales', 'Name'], how = 'all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [149]:
df5.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [148]:
df5.na.fill('Fav. name').show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|Fav. name| null|
|emp3|Fav. name|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



Spark figures out that the fav. name would be for filling the missing values in the name column, because that's the column with string. But it's not always safe to rely on that. Use subset to assign the columns to be filled. 

In [150]:
df5.na.fill('fav. name', subset=['Name']).show()

+----+---------+-----+
|  Id|     Name|Sales|
+----+---------+-----+
|emp1|     John| null|
|emp2|fav. name| null|
|emp3|fav. name|345.0|
|emp4|    Cindy|456.0|
+----+---------+-----+



** Filling in using the mean value **

In [151]:
mean_sales = df5.agg({'Sales':'mean'})

In [154]:
mean_sales.collect()

[Row(avg(Sales)=400.5)]

In [155]:
mean_sales.collect()[0]

Row(avg(Sales)=400.5)

In [157]:
mean_sales_val = mean_sales.collect()[0][0]

In [160]:
df5.na.fill(mean_sales_val, subset = ['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Date and timestamps

Use the appl_stock data used earlier, stored in the df3 data frame. 

In [162]:
df3.show(5)

+--------------------+----------+----------+------------------+------------------+---------+------------------+
|                Date|      Open|      High|               Low|             Close|   Volume|         Adj Close|
+--------------------+----------+----------+------------------+------------------+---------+------------------+
|2010-01-04 00:00:...|213.429998|214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05 00:00:...|214.599998|215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06 00:00:...|214.379993|    215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07 00:00:...|    211.75|212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08 00:00:...|210.299994|212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
+--------------------+----------+----------+------------------+------------------+---------+------------

In [163]:
df3.head(2)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date=datetime.datetime(2010, 1, 5, 0, 0), Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002)]

In [167]:
df3.select(['Date', 'Open', 'Close']).show(6)

+--------------------+------------------+------------------+
|                Date|              Open|             Close|
+--------------------+------------------+------------------+
|2010-01-04 00:00:...|        213.429998|        214.009998|
|2010-01-05 00:00:...|        214.599998|        214.379993|
|2010-01-06 00:00:...|        214.379993|        210.969995|
|2010-01-07 00:00:...|            211.75|            210.58|
|2010-01-08 00:00:...|        210.299994|211.98000499999998|
|2010-01-11 00:00:...|212.79999700000002|210.11000299999998|
+--------------------+------------------+------------------+
only showing top 6 rows



In [168]:
from pyspark.sql.functions import (dayofmonth, hour, dayofyear, month, year, 
                                   weekofyear, format_number, date_format)

In [179]:
df3.select([format_number(df3['Open'], 2).alias('Open'), format_number(df3['Close'], 2)]).show(10)

+------+-----------------------+
|  Open|format_number(Close, 2)|
+------+-----------------------+
|213.43|                 214.01|
|214.60|                 214.38|
|214.38|                 210.97|
|211.75|                 210.58|
|210.30|                 211.98|
|212.80|                 210.11|
|209.19|                 207.72|
|207.87|                 210.65|
|210.11|                 209.43|
|210.93|                 205.93|
+------+-----------------------+
only showing top 10 rows



Let's say we want the average closing price per year. 

In [190]:
df31 = (df3.withColumn('Year', year(df3['Date']))).select(['Year', 'Close'])

In [191]:
df31.show(3)

+----+----------+
|Year|     Close|
+----+----------+
|2010|214.009998|
|2010|214.379993|
|2010|210.969995|
+----+----------+
only showing top 3 rows



In [196]:
df32 = df31.groupBy('Year').mean().select(['Year', 'avg(Close)'])

In [199]:
df32.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [204]:
df32.withColumnRenamed('avg(Close)', 'mean_close').orderBy('Year').show()

+----+------------------+
|Year|        mean_close|
+----+------------------+
|2010| 259.8424600000002|
|2011|364.00432532142867|
|2012| 576.0497195640002|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2015|120.03999980555547|
|2016|104.60400786904763|
+----+------------------+



In [206]:
df33 = df32.withColumnRenamed('avg(Close)', 'mean_close').orderBy('Year')

In [209]:
df33.select(['Year', format_number(df33['mean_close'], 2).alias('mean_close')]).show()

+----+----------+
|Year|mean_close|
+----+----------+
|2010|    259.84|
|2011|    364.00|
|2012|    576.05|
|2013|    472.63|
|2014|    295.40|
|2015|    120.04|
|2016|    104.60|
+----+----------+

